In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.tools.render import format_tool_to_openai_function
from langchain.prompts import ChatPromptTemplate
from utils import get_openai_api_key
from Tools.temperature import get_current_temperature
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.agent import AgentFinish
import openai

openai.api_key = get_openai_api_key()
functions = [
        format_tool_to_openai_function(f) for f in [get_current_temperature]
]
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant. If user ask you about movie, you should answer sincerely. \
        if you don't have information of user, suppose that the user is in Seoul and don't ask user"),
    ("user", "{input}"),
])

def route(result):
	if isinstance(result, AgentFinish): # 함수를 쓰지 않기로 결정한다면 -> content 반환
		return result.return_values['output']
	else: # 함수를 쓰기로 결정한다면 -> 함수명에 따라 어떤 함수를 쓸지 결정해주고, argument를 넣은 값 반환
		tools = {
			"get_current_temperature": get_current_temperature,
		}
		return tools[result.tool].run(result.tool_input)
  
model = ChatOpenAI(temperature=0).bind(functions=functions)
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route
chain.invoke({"input":"오늘 부산 날씨는 어떤것 같아"})

'The current temperature is -3.5°C'

In [3]:
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You are a movie recommender. Use your knowledge to recommend movies that fit\
        and rewrite with given informations {informations}. You shoud answer using only Korean and markdown structure"),
])

In [4]:
model2 = ChatOpenAI(temperature=0, max_tokens=1024)
chain2 = prompt2 | model2

In [7]:
result1 = informations=chain.invoke({"input":"오늘 날씨에 맞는 영화 추천좀"})
result2 = chain2.invoke({"informations": result1})

In [9]:
print(result2.content)

현재 온도는 -8.2°C입니다. 추운 날씨에는 온기를 느낄 수 있는 영화를 추천해드릴게요. 아래의 영화들을 즐기시면 추위를 잊고 영화 속에서 따뜻한 분위기를 느낄 수 있을 거예요:

1. 겨울왕국 (Frozen): 겨울 풍경과 따뜻한 가족 사랑을 다룬 애니메이션 영화입니다. 눈과 얼음으로 뒤덮인 왕국에서 펼쳐지는 이야기는 추운 날씨에도 마음을 녹일 수 있어요.

2. 러브 액츄얼리 (Love Actually): 크리스마스 분위기와 사랑 이야기가 어우러진 영화입니다. 다양한 인물들의 이야기가 따뜻한 감동을 전해줄 거예요.

3. 캐롤 (Carol): 1950년대 겨울을 배경으로 한 로맨스 드라마입니다. 눈이 내리는 도시에서 펼쳐지는 사랑 이야기는 추운 날씨에 어울리는 영화 중 하나에요.

4. 홈 애로인 (Home Alone): 겨울 방학 동안 집에 혼자 남겨진 소년의 모험을 그린 코미디 영화입니다. 따뜻한 가족의 사랑과 유쾌한 이야기로 추위를 잊게 해줄 거예요.

이 영화들을 시청하면 추운 날씨에도 따뜻한 감정을 느낄 수 있을 거예요. 즐겁게 영화를 감상하세요!


In [181]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import random
import time

# Selenium 웹 드라이버 설정 
driver = webdriver.Chrome()
options = Options()

#지정한 user-agent로 설정합니다.
user_agent = "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
options.add_argument('user-agent=' + user_agent)
options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.

# 웹 페이지 열기
url = 'https://movie.daum.net/premovie/theater'
driver.get(url)

# 페이지가 완전히 로딩되도록 3초동안 기다림
time.sleep(3)

# 해당 경로의 요소 가져오기
ol_element = driver.find_element(By.XPATH, '//*[@id="mainContent"]/div/div[2]/ol')
# ol 아래의 모든 li 요소 가져오기
li_elements = ol_element.find_elements(By.TAG_NAME, 'li')
infos = []

# 가져온 li 요소 출력
for li in li_elements:
    li.click()
    # 0.4에서 0.6 사이의 랜덤한 값을 선택하여 sleep에 적용
    random_sleep_time = random.uniform(0.4, 0.6)
    time.sleep(random_sleep_time)
    infos.append(li.text)

# 웹 드라이버 종료
driver.quit()

In [182]:
infos

['12세이상관람가\n임진왜란 발발로부터 7년이 지난 1598년 12월. 이순신(김윤석)은 왜군의 수장이던 도요토미 히데요시가 갑작스럽게 사망한 뒤 왜군들이 조선에서 황급히 퇴각하려 한다는 것을 알게 된다. “절대 이렇게 전쟁을 끝내서는 안 된다” 왜군을 완벽하게 섬멸하는 것이 이 전쟁을 올바르게 끝나는 것이라 생각한 이순신은 명나라와 조명연합함대를 꾸려 왜군의 퇴각로를 막고 적들을 섬멸하기로 결심한다.하지만 왜군의 뇌물 공세에 넘어간 명나라 도독 진린(정재영)은 왜군에게 퇴로를 열어주려 하고,설상가상으로 왜군 수장인 시마즈(백윤식)의 살마군까지 왜군의 퇴각을 돕기 위해 노량으로 향하는데…2023년 12월, 모두를 압도할 최후의 전투가 시작된다!\n노량: 죽음의 바다\n평점8.4 예매율43.1%\n개봉23.12.20',
 '12세이상관람가\n1979년 12월 12일, 수도 서울 군사반란 발생그날, 대한민국의 운명이 바뀌었다대한민국을 뒤흔든 10월 26일 이후, 서울에 새로운 바람이 불어온 것도 잠시12월 12일, 보안사령관 전두광이 반란을 일으키고군 내 사조직을 총동원하여 최전선의 전방부대까지 서울로 불러들인다.권력에 눈이 먼 전두광의 반란군과 이에 맞선 수도경비사령관 이태신을 비롯한진압군 사이, 일촉즉발의 9시간이 흘러가는데…목숨을 건 두 세력의 팽팽한 대립오늘 밤, 대한민국 수도에서 가장 치열한 전쟁이 펼쳐진다!\n서울의 봄\n평점9.5 예매율25.9%\n개봉23.11.22',
 '12세이상관람가\n아틀란티스 왕국을 이끌 왕의 자리에 오른 ‘아쿠아맨’. 그 앞에 ‘블랙 만타’가 세상을 뒤흔들 강력한 지배 아이템 ‘블랙 트라이던트’를 손에 넣게 된다.그동안 겪지 못 했던 최악의 위협 속 ‘아쿠아맨’은‘블랙 만타’와 손을 잡았던 이부 동생 ‘옴’ 없이는 절대적 힘이 부족한 상황.바다를 지배할 슈퍼 히어로가 세상의 판도를 바꾼다! \n아쿠아맨과 로스트 킹덤\n평점6.3 예매율9.4%\n개봉23.12.20',
 '전체관람가\n최초의 3D CG! 제작 기간 7년 최

In [195]:
import re
from datetime import datetime
import pandas as pd

def parse_movie_info(movie_info_list):
    parsed_info = []
    for info in movie_info_list:
        split_info = info.split('\n')
        if len(split_info) == 5:
            if len(split_info[3].split())!=2:
                continue
            age_limit = split_info[0]
            plot = split_info[1]
            title = split_info[2]
            rating, booking_rate = (split_info[3].split())
            # rating = re.sub(r'\D', '',rating)
            # booking_rate = re.sub(r'\D', '',booking_rate)            
            release_date = re.sub(r'\D', '', split_info[4])
            release_date_formatted = datetime.strptime('20'+release_date, '%Y%m%d').strftime('%y년 %m월 %d일')
            
            parsed_info.append({
                '관람가': age_limit,
                '제목' : title,
                '평점': rating,
                '예매율': booking_rate,
                '개봉일': release_date_formatted,
                '줄거리': plot,
            })
    
    return pd.DataFrame(parsed_info)
parse_movie_info(infos)

,관람가,제목,평점,예매율,개봉일,줄거리
0,12세이상관람가,노량: 죽음의 바다,평점8.4,예매율43.1%,23년 12월 20일,임진왜란 발발로부터 7년이 지난 1598년 12월. 이순신(김윤석)은 왜군의 수장이...
1,12세이상관람가,서울의 봄,평점9.5,예매율25.9%,23년 11월 22일,"1979년 12월 12일, 수도 서울 군사반란 발생그날, 대한민국의 운명이 바뀌었다..."
2,12세이상관람가,아쿠아맨과 로스트 킹덤,평점6.3,예매율9.4%,23년 12월 20일,아틀란티스 왕국을 이끌 왕의 자리에 오른 ‘아쿠아맨’. 그 앞에 ‘블랙 만타’가 세...
3,전체관람가,신차원! 짱구는 못말려 더 무비 초능력 대결전 ~날아라 수제김밥~,평점10.0,예매율6.7%,23년 12월 22일,최초의 3D CG! 제작 기간 7년 최고의 웃음과 감동! 최강의 스케일 옷까지 갈아...
4,전체관람가,트롤: 밴드 투게더,평점10.0,예매율5.9%,23년 12월 20일,최애 아이돌 그룹 재결합 초읽기! 우리 함께하면 더 좋잖아! 전 세계 모든 트롤을 ...
5,전체관람가,뽀로로 극장판 슈퍼스타 대모험,평점10.0,예매율2.5%,23년 12월 13일,뽀로로와 친구들은 전 우주를 통틀어 최고의 슈퍼스타를 뽑는 음악 축제 ‘파랑돌 슈퍼...
6,12세이상관람가,괴물,평점8.0,예매율1.0%,23년 11월 29일,“우리 동네에는 괴물이 산다” 싱글맘 사오리(안도 사쿠라)는 아들 미나토(쿠로카와 ...
7,전체관람가,크레센도,평점10.0,예매율0.6%,23년 12월 20일,"“예술성, 테크닉, 기교, 드라마, 짜릿함. 그는 피아노 연주의 극치를 보여주었다!..."
8,12세이상관람가,3일의 휴가,평점7.8,예매율0.5%,23년 12월 06일,“따님은 어머님을 보거나 목소리를 들을 수 없고요. 그냥 따님의 행복한 기억만 담고...
9,전체관람가,말하고 싶은 비밀,평점9.4,예매율0.3%,23년 12월 13일,어느 날 갑자기 고백 사고!? 마음이 잘못 배달되었다! ‘노조미’는 자신의 책상 서...
